In [191]:
import json

with open('Data/advanced_fpl_cleaned_data.json', 'r') as json_data:
    data = json.load(json_data)

In [195]:
#Index is (4-player["element_type"]) Forward, midfield, defender, goalie
togga_scoring = {
"goals_scored": [9,9,10,10],
"assists": [6,6,8,9],
"keyPass_total": [2,2,2,6],
"ontarget_scoring_att": [2,2,2,2],
"accurateCrosses_total": [1,1,1,1],
"successful_dribbles": [1,1,1,1],
"dispossessed": [-0.5,-0.5,-0.5,-0.5],
"own_goals": [-9,-9,-9,-9],
"clean_sheets": [0,1,6,8],
"saves": [0,0,0,2],
"interception_total": [1,1,1,1],
"tackle_total": [1,1,1,1],
"goals_conceded": [0,0,-2,-2],
"penalties_saved": [0,0,0,8],
"yellow_cards": [-3,-3,-3,-3],
"red_cards": [-7,-7,-7,-7],
"aerialWonPerGame_total": [0.5,0.5,1,1],
"interception_total": [0,0,0.25,0.25]}

In [196]:
by_position = {1:[],2:[],3:[],4:[]}
for player in data["elements"]:
    index = 4-player["element_type"]
    for season in player["history"]:
        score = 0
        for stat in togga_scoring:
            if player["history"][season].get(stat):
                score += togga_scoring[stat][index]*player["history"][season][stat]
                player["history"][season][stat + "_score"] = togga_scoring[stat][index]*player["history"][season][stat]
            else:
                player["history"][season].update({stat:-1})
        player["history"][season]["togga_score"] = score
    # Set the scores to 0 if they have no history
    if not (player["history"].get("2015/16")):
        player['history'].update({'2015/16': {"togga_score":0}})
    if not (player["history"].get("2014/15")):
        player['history'].update({'2014/15': {"togga_score":0}})
    if not (player["history"].get("2013/14")):
        player['history'].update({'2013/14': {"togga_score":0}})
    if not (player["history"].get("2012/13")):
        player['history'].update({'2012/13': {"togga_score":0}})
    by_position[player["element_type"]].append(player)

In [197]:
with open('Data/example_player.json', 'w') as json_data:
    json.dump(data["elements"][81], json_data)

In [102]:
with open('Data/advanced_fpl_cleaned_data.json', 'w') as json_data:
    json.dump(data, json_data)

In [21]:
with open('Data/positional.json', 'w') as json_data:
    json.dump(by_position, json_data)

In [187]:
bpcopy = by_position.copy()
for position in by_position:
    for i, player in enumerate(by_position[position]):
        for year in ['2016/17', '2015/16', '2014/15', '2013/14', '2012/13']:
            new_stats={}
            for stat in player['history'][year]:
                if stat in togga_scoring or stat in ['togga_score', 'apps', 'start_cost', 'end_cost', 
                                                    'total_pass', 'minutes']:
                    new_stats.update({stat: player['history'][year][stat]})  
            bpcopy[position][i]['history'].update({year:new_stats.copy()})
                
for year in ['2016/17', '2015/16', '2014/15', '2013/14', '2012/13']:
    g = []
    d = []
    m = []
    f = []
    for i, player in enumerate(bpcopy[1]):
        if player['history'][year]['togga_score'] != 0:
            copy = player['history'][year].copy()
            copy['name'] = player["name"]
            g.append(copy)
    for i, player in enumerate(bpcopy[2]):
        if player['history'][year]['togga_score'] != 0:
            copy = player['history'][year].copy()
            copy['name'] = player["name"]
            d.append(copy)
    for i, player in enumerate(bpcopy[3]):
        if player['history'][year]['togga_score'] != 0:
            copy = player['history'][year].copy()
            copy['name'] = player["name"]
            m.append(copy)
    for i, player in enumerate(bpcopy[4]):
        if player['history'][year]['togga_score'] != 0:
            copy = player['history'][year].copy()
            copy['name'] = player["name"]
            f.append(copy)
    sorted_goalies = sorted(g, key=lambda k: k['togga_score'], reverse=True)
    with open('Data/Sorted_Positions/sorted_goalies_%s+%s.json' % (year[:4], year[-2:]), 'w') as json_data:
        json.dump(sorted_goalies, json_data)
    sorted_defense = sorted(d, key=lambda k: k['togga_score'], reverse=True)
    with open('Data/Sorted_Positions/sorted_defense_%s+%s.json' % (year[:4], year[-2:]), 'w') as json_data:
        json.dump(sorted_defense, json_data)
    sorted_midfield = sorted(m, key=lambda k: k['togga_score'], reverse=True)
    with open('Data/Sorted_Positions/sorted_midfield_%s+%s.json' % (year[:4], year[-2:]), 'w') as json_data:
        json.dump(sorted_midfield, json_data)
    sorted_forwards = sorted(f, key=lambda k: k['togga_score'], reverse=True)
    with open('Data/Sorted_Positions/sorted_forwards_%s+%s.json' % (year[:4], year[-2:]), 'w') as json_data:
        json.dump(sorted_forwards, json_data)

In [97]:
for i in range(10):
    name=sorted_goalies[i]['name']
    score=sorted_goalies[i]['history']['2012/13']['togga_score']
    print("name: %s\tscore:%f" % (name, score))

name: Simon Mignolet	score:312.250000
name: Petr Cech	score:302.750000
name: Asmir Begovic	score:258.000000
name: Joe Hart	score:251.250000
name: David de Gea	score:214.000000
name: Ben Foster	score:195.000000
name: Michel Vorm	score:184.000000
name: Bradley Guzan	score:174.000000
name: Hugo Lloris	score:156.500000
name: Adam Federici	score:124.000000


In [69]:
for i in range(10):
    name=sorted_defense[i]['name']
    score=sorted_defense[i]['history']['2016/17']['togga_score']
    print("name: %s\tscore:%f" % (name, score))

name: Marcos Alonso	score:470.000000
name: Leighton Baines	score:350.250000
name: Kyle Walker	score:344.500000
name: José Holebas	score:319.500000
name: Christian Fuchs	score:316.750000
name: Chris Brunt	score:311.500000
name: Ryan Bertrand	score:307.000000
name: César Azpilicueta	score:305.750000
name: Gary Cahill	score:292.750000
name: Matthew Lowton	score:290.500000


In [70]:
for i in range(10):
    name=sorted_midfield[i]['name']
    score=sorted_midfield[i]['history']['2016/17']['togga_score']
    print("name: %s\tscore:%f" % (name, score))

name: Christian Eriksen	score:779.500000
name: Gylfi Sigurdsson	score:739.000000
name: Alexis Sánchez	score:583.000000
name: Mesut Özil	score:582.500000
name: James Milner	score:581.500000
name: Eden Hazard	score:570.500000
name: Philippe Coutinho	score:543.500000
name: Andros Townsend	score:500.500000
name: Ross Barkley	score:483.500000
name: Kevin De Bruyne	score:471.500000


In [71]:
for i in range(10):
    name=sorted_forwards[i]['name']
    score=sorted_forwards[i]['history']['2016/17']['togga_score']
    print("name: %s\tscore:%f" % (name, score))

name: Romelu Lukaku	score:543.500000
name: Harry Kane	score:529.000000
name: Christian Benteke	score:419.000000
name: Sergio Agüero	score:408.000000
name: Zlatan Ibrahimovic	score:402.500000
name: Jamie Vardy	score:315.500000
name: Fernando Llorente	score:310.500000
name: Troy Deeney	score:293.000000
name: Sam Vokes	score:288.000000
name: Álvaro Negredo	score:284.000000


In [90]:
2015+a = sorted(by_position[4], key=lambda k: k['history']['2015/16']['togga_score'], reverse=True)

TypeError: unhashable type: 'list'

In [31]:
by_year = {1:[],2:[],3:[],4:[],5:[]}
for i, player in enumerate(data["elements"]):
    by_year[1].append(player)
    if(len(player["history"])>1) and player["history"].get("2015/16"):
        by_year[2].append(player)
        if(len(player["history"])>2) and player["history"].get("2014/15"):
            by_year[3].append(player)
            if(len(player["history"])>3) and player["history"].get("2013/14"):
                by_year[4].append(player)
                if(len(player["history"])>4) and player["history"].get("2012/13"):
                    by_year[5].append(player)

In [27]:
with open('Data/by_experience.json', 'w') as json_data:
    json.dump(by_year, json_data)

In [62]:
point_ranking_16_17 = []
point_ranking_15_16 = []
point_ranking_14_15 = []
point_ranking_13_14 = []
point_ranking_12_13 = []
for i, player in enumerate(data["elements"]):
    try:
        if player in by_year[1]:
            point_ranking_16_17.append((player["name"], player["history"]["2016/17"]["togga_score"]))
        if player in by_year[2]:
            point_ranking_15_16.append((player["name"], player["history"]["2015/16"]["togga_score"]))
        if player in by_year[3]:
            point_ranking_14_15.append((player["name"], player["history"]["2014/15"]["togga_score"]))
        if player in by_year[4]:
            point_ranking_13_14.append((player["name"], player["history"]["2013/14"]["togga_score"]))
        if player in by_year[5]:
            point_ranking_12_13.append((player["name"], player["history"]["2012/13"]["togga_score"]))
    except:
        print(i)


In [46]:
#print(sorted(point_ranking_16_17, key=lambda tup: tup[1], reverse=True))
#print(sorted(point_ranking_15_16, key=lambda tup: tup[1], reverse=True))
#print(sorted(point_ranking_14_15, key=lambda tup: tup[1], reverse=True))
#print(sorted(point_ranking_13_14, key=lambda tup: tup[1], reverse=True))
#print(sorted(point_ranking_12_13, key=lambda tup: tup[1], reverse=True))